# AI Coding Partner Agent 🤖💻

A demonstration of advanced Agentic AI principles: **Iterative Reasoning** and **Tool Use**.

This agent tackles coding tasks by using a ReAct (Reason-Act) loop:
1.  **Reason:** Plans a solution and writes Python code.
2.  **Act:** Executes the code in a safe environment (its tool).
3.  **Observe:** Analyzes the output or error.
4.  **Loop:** Repeats steps 1-3 until the task is solved or a limit is reached.

This showcases an AI's ability to autonomously debug and problem-solve, a core capability for an AI employee.

In [1]:
# Install the required package for Google's Gemini API
!pip install google-generativeai

# Import the necessary libraries
import google.generativeai as genai
from google.colab import userdata
from textwrap import fill

In [2]:
# @title Step 1: Setup and Authentication
from google.colab import userdata
import google.generativeai as genai

# Read the Gemini API key from Colab's Secrets
# Click the key icon (🔑) in the left sidebar, then "Add new secret".
# Name it 'GEMINI_API_KEY' and paste your key from https://aistudio.google.com/
google_api_key = userdata.get('GEMINI_API_KEY')

# Configure the Gemini client
genai.configure(api_key=google_api_key)

# Initialize the model. We'll use the fast and capable 'gemini-1.5-flash'
model = genai.GenerativeModel('gemini-1.5-flash')

print("✅ Gemini model initialized successfully!")

✅ Gemini model initialized successfully!


In [3]:
# @title Step 2: Create the Code Execution Tool
def execute_python_code(code_string):
    """
    A safe-ish function to execute a string of Python code in the Colab environment.
    This acts as the agent's "tool" for running code and seeing the output.
    """
    try:
        # Create a dedicated local namespace for execution to avoid polluting the global scope.
        local_namespace = {}
        # Execute the code
        exec(code_string, globals(), local_namespace)
        # Try to capture any output that was printed or stored in an 'output' variable
        output = local_namespace.get('output', 'Code executed successfully. No output captured.')
        return f"SUCCESS: Output: {output}"
    except Exception as e:
        # Return the error message so the agent can learn from it
        return f"ERROR: {type(e).__name__}: {e}"

In [4]:
# @title Step 3: Build the Coding Agent
def coding_agent(task_description, max_attempts=4):
    """
    An AI agent that writes and iteratively debugs code to complete a task.
    """
    print(f"💻 Task received: {task_description}")
    print("🧠 Thinking...")

    # We'll give the model a very clear system prompt
    system_prompt = """
    You are an expert Python coding AI. Your goal is to write code to solve a user's task.
    You must follow these steps:
    1. Reason about what code needs to be written.
    2. Write the code inside a markdown code block labeled 'python'.
    3. The code must print its final result to a variable called 'output'.
    4. You will then be shown the execution result or error.
    5. If there is an error, analyze it, fix the code, and try again.

    You must only respond with the code. Do not respond with any other text or explanations.
    """

    # Start a conversation history
    conversation = [
        {"role": "user", "parts": [system_prompt + f"\nUser's Task: {task_description}"]},
    ]

    # Allow for a few attempts in case of errors
    for attempt in range(max_attempts):
        print(f"\n🔄 Attempt {attempt + 1} of {max_attempts}")
        print("-" * 30)

        # Generate code from the model
        response = model.generate_content(conversation)
        generated_code = response.text
        # Clean the output to extract just the Python code
        if "```python" in generated_code:
            generated_code = generated_code.split("```python")[1].split("```")[0].strip()
        elif "```" in generated_code:
            generated_code = generated_code.split("```")[1].split("```")[0].strip()

        print(f"📝 Generated code:\n{generated_code}")

        # Execute the generated code (Use the Tool!)
        execution_result = execute_python_code(generated_code)
        print(f"⚡ {execution_result}")

        # Check if the execution was successful
        if execution_result.startswith("SUCCESS"):
            print("\n✅ Task completed successfully!")
            return f"**Final Code:**\n```python\n{generated_code}\n```\n\n**Result:**\n{execution_result}"

        # If it failed, add the error to the conversation so the model can learn from it
        conversation.append({"role": "model", "parts": [f"Code I wrote:\n{generated_code}\n\nResult:\n{execution_result}"]})
        conversation.append({"role": "user", "parts": ["That code failed. Please analyze the error and write new, fixed code."]})

    return "❌ The agent was unable to complete the task after several attempts."

In [6]:
# @title Step 4: Run the Agent!
# Define your task here. Try this one first, then experiment with others!
task = "Write a function that reverses a string without using any built-in functions like [::-1] or reversed(). Test it with the string 'hello'."
final_output = coding_agent(task)

print("\n" + "="*60)
print("FINAL OUTPUT")
print("="*60)
print(final_output)

💻 Task received: Write a function that reverses a string without using any built-in functions like [::-1] or reversed(). Test it with the string 'hello'.
🧠 Thinking...

🔄 Attempt 1 of 4
------------------------------
📝 Generated code:
def reverse_string(s):
    reversed_str = ""
    for i in range(len(s) - 1, -1, -1):
        reversed_str += s[i]
    return reversed_str

output = reverse_string("hello")
⚡ SUCCESS: Output: olleh

✅ Task completed successfully!

FINAL OUTPUT
**Final Code:**
```python
def reverse_string(s):
    reversed_str = ""
    for i in range(len(s) - 1, -1, -1):
        reversed_str += s[i]
    return reversed_str

output = reverse_string("hello")
```

**Result:**
SUCCESS: Output: olleh


## Summary & Evaluation

**Success Criteria Met:**

✅ **Tool Use:** The agent successfully used the Python `exec()` function as a tool to execute its generated code.

✅ **Iterative Reasoning (ReAct Loop):** The agent's architecture is built on a loop of reasoning, acting, and observing. It successfully solved the prime number task on its first attempt.

✅ **Autonomous Problem-Solving:** The agent received a high-level goal and completed it without any human intervention.

**This project demonstrates a more advanced form of Agentic AI than the Research Agent, showcasing an AI's ability to not just use tools, but to create and validate its own solutions—a critical skill for an AI employee.**